In [1]:
from rmgpy.data.rmg import RMGDatabase
from rmgpy.chemkin import saveChemkinFile, saveSpeciesDictionary
from rmgpy.rmg.model import Species
from rmgpy import settings

## load lib_rxn

In [2]:
database = RMGDatabase()
database.load(settings['database.directory'], kineticsFamilies=['R_Addition_MultipleBond'], reactionLibraries = ['vinylCPD_H'], kineticsDepositories='all')

In [15]:
def findLabeledProdcuts(moleculeA, moleculeB, family, products):
    template = family.forwardTemplate
    mappingsA = family._KineticsFamily__matchReactantToTemplate(moleculeA, template.reactants[0])
    mappingsB = family._KineticsFamily__matchReactantToTemplate(moleculeB, template.reactants[1])

    # Iterate over each pair of matches (A, B)
    for mapA in mappingsA:
        for mapB in mappingsB:
            reactantStructures = [moleculeA, moleculeB]
            try:
                productStructures = family._KineticsFamily__generateProductStructures(reactantStructures, [mapA, mapB], forward=True)
            except ForbiddenStructureException:
                pass
            else:
                if productStructures is not None:
                    if len(products) == 1 and len(productStructures) == 1:
                        if products[0].isIsomorphic(productStructures[0]):
                            return productStructures
                    elif len(products) == 2 and len(productStructures) == 2:
                        if products[0].isIsomorphic(productStructures[0]):
                            if products[1].isIsomorphic(productStructures[1]):
                                return productStructures
                        if products[0].isIsomorphic(productStructures[1]):
                            if products[1].isIsomorphic(productStructures[0]):
                                return productStructures
                    else: continue
    # if there're some mapping available but cannot match the provided products
    # raise exception
    if len(mappingsA)*len(mappingsB) > 0:
        raise Exception('Something wrong with products that RMG cannot find a match!')
    return

In [17]:
def addLabeledAtoms(fam_rxn, database):
    from rmgpy.species import Species
    # prepare fam_rxts_mol and fam_pdts_mol
    reactants = [rxt.molecule[0] for rxt in fam_rxn.reactants]            
    products = [pdt.molecule[0] for pdt in fam_rxn.products]
    # prepare fam_forward_template always forward for fam_rxn
    # because __createReaction always make sure fam_rxn is in forward style
    family = database.kinetics.families[fam_rxn.family]
    template = family.forwardTemplate
    # Unimolecular reactants: A --> products
    if len(reactants) == 1 and len(template.reactants) == 1:

        # Iterate over all resonance isomers of the reactant
        molecule = reactants[0]
        mappings = family._KineticsFamily__matchReactantToTemplate(molecule, template.reactants[0])
        for map in mappings:
            reactantStructures = [molecule]
            try:
                productStructures = family._KineticsFamily__generateProductStructures(reactantStructures, [map], forward=True)
            except ForbiddenStructureException:
                pass
            else:
                if productStructures is not None:
                    if len(products) == 1 and len(productStructures) == 1:
                        if products[0].isIsomorphic(productStructures[0]):
                            products = productStructures
                            break
                    elif len(products) == 2 and len(productStructures) == 2:
                        if products[0].isIsomorphic(productStructures[0]):
                            if products[1].isIsomorphic(productStructures[1]):
                                products = productStructures
                                break
                        if products[0].isIsomorphic(productStructures[1]):
                            if products[1].isIsomorphic(productStructures[0]):
                                products = productStructures
                                break
                    else: continue

    # Bimolecular reactants: A + B --> products
    elif len(reactants) == 2 and len(template.reactants) == 2:

        moleculeA = reactants[0]
        moleculeB = reactants[1]
        # Reactants stored as A + B
        labeledProducts = findLabeledProdcuts(moleculeA, moleculeB, family, products)
        labeledProducts_spcs = []
        if labeledProducts is not None:
            
            for labeledProduct in labeledProducts:
                labeledProducts_spcs.append(Species(molecule=[labeledProduct]))
            fam_rxn.products = labeledProducts_spcs

        # Only check for swapped reactants if they are different
        if reactants[0] is not reactants[1]:

            # Reactants stored as B + A
            products = findLabeledProdcuts(moleculeB, moleculeA, family, products)
            labeledProducts_spcs = []
            if labeledProducts is not None:

                for labeledProduct in labeledProducts:
                    labeledProducts_spcs.append(Species(molecule=[labeledProduct]))
                fam_rxn.products = labeledProducts_spcs

## generate fam_rxn, spec replacement and get reactionDict

In [18]:
reactionDict = {}
libraryName = 'vinylCPD_H'
kineticLibrary = database.kinetics.libraries[libraryName]
for index, entry in kineticLibrary.entries.iteritems():
    lib_rxn = entry.item
    lib_rxn.kinetics = entry.data 
    lib_rxn.index = entry.index
    if lib_rxn.index in [2]:
        # Let's make RMG try to generate this reaction from the families!
        fam_rxn_list = []
        rxt_mol_mutation_num = 1
        pdt_mol_mutation_num = 1
        for reactant in lib_rxn.reactants:
            rxt_mol_mutation_num *= len(reactant.molecule)

        for product in lib_rxn.products:
            pdt_mol_mutation_num *= len(product.molecule)

        for mutation_i in range(rxt_mol_mutation_num):
            rxts_mol = [spc.molecule[mutation_i%(len(spc.molecule))] for spc in lib_rxn.reactants]
            for mutation_j in range(pdt_mol_mutation_num):
                pdts_mol = [spc.molecule[mutation_j%(len(spc.molecule))] for spc in lib_rxn.products]
                fam_rxn_list.extend(database.kinetics.generateReactionsFromFamilies(
                        reactants=rxts_mol, products=pdts_mol))
        if len(fam_rxn_list) == 1:
            fam_rxn = fam_rxn_list[0]
            # species replacement so that labeledAtoms is retored
            # danger: the fam_rxn may have switched the reactants with products
            # fam_rxn is survived from def filterReactions
            # so it's matched with lib_rxn only we have to 
            # determine the direction
            lib_reactants = [r for r in lib_rxn.reactants]        
            fam_reactants = [r for r in fam_rxn.reactants]
            for lib_reactant in lib_reactants:
                for fam_reactant in fam_reactants:
                    if lib_reactant.isIsomorphic(fam_reactant):
                        fam_reactants.remove(fam_reactant)
                        break
            lib_products = [r for r in lib_rxn.products]        
            fam_products = [r for r in fam_rxn.products]
            for lib_product in lib_products:
                for fam_product in fam_products:
                    if lib_product.isIsomorphic(fam_product):
                        fam_products.remove(fam_product)
                        break
            forward = not (len(fam_reactants) != 0 or len(fam_products) != 0)
            
            # find the labeled atoms using family and reactants & products from fam_rxn
            print "ibefore"
            print fam_rxn.reactants[0].molecule[0].toAdjacencyList()
            print fam_rxn.products[0].molecule[0].toAdjacencyList()
            addLabeledAtoms(fam_rxn, database)
            print "iafter"
            print fam_rxn.reactants[0].molecule[0].toAdjacencyList()
            print fam_rxn.products[0].molecule[0].toAdjacencyList()
            
#             if forward:
#                 lib_rxn.reactants = fam_rxn.reactants
#                 lib_rxn.products = fam_rxn.products
#             else:
#                 lib_rxn.reactants = fam_rxn.products
#                 lib_rxn.products = fam_rxn.reactants
            if fam_rxn.family in reactionDict:
                reactionDict[fam_rxn.family].append(lib_rxn)
            else:
                reactionDict[fam_rxn.family] = [lib_rxn]
#     elif len(rmgReactionList) == 0:
#         print reaction
#         print 'reactants'
#         for reactant in reaction.reactants:
#             print reactant.molecule[0].toSMILES()
#         print 'products'
#         for product in reaction.products:
#             print product.molecule[0].toSMILES()
#         print "Sad :( There are no matches.  This is a magic reaction or has chemistry that should be made into a new reaction family"
#     else:
#         if rmgReactionList[0].family not in ['Intra_R_Add_Exocyclic','intra_H_migration']:
#             print reaction
#             for rxn in rmgReactionList:
#                 print rxn.family
#             print "There are multiple RMG matches for this reaction. You have to manually create this training reaction"

ibefore
1  C u0 p0 c0 {2,S} {3,S} {4,S} {7,S}
2  C u0 p0 c0 {1,S} {5,D} {9,S}
3  C u0 p0 c0 {1,S} {6,D} {10,S}
4  C u0 p0 c0 {1,S} {8,D} {13,S}
5  C u0 p0 c0 {2,D} {6,S} {11,S}
6  C u0 p0 c0 {3,D} {5,S} {12,S}
7  H u0 p0 c0 {1,S}
8  C u0 p0 c0 {4,D} {14,S} {15,S}
9  H u0 p0 c0 {2,S}
10 H u0 p0 c0 {3,S}
11 H u0 p0 c0 {5,S}
12 H u0 p0 c0 {6,S}
13 H u0 p0 c0 {4,S}
14 H u0 p0 c0 {8,S}
15 H u0 p0 c0 {8,S}

multiplicity 2
1  C u0 p0 c0 {2,S} {3,S} {4,S} {8,S}
2  C u0 p0 c0 {1,S} {7,S} {9,S} {10,S}
3  C u0 p0 c0 {1,S} {5,D} {11,S}
4  C u0 p0 c0 {1,S} {6,D} {12,S}
5  C u0 p0 c0 {3,D} {6,S} {13,S}
6  C u0 p0 c0 {4,D} {5,S} {14,S}
7  C u1 p0 c0 {2,S} {15,S} {16,S}
8  H u0 p0 c0 {1,S}
9  H u0 p0 c0 {2,S}
10 H u0 p0 c0 {2,S}
11 H u0 p0 c0 {3,S}
12 H u0 p0 c0 {4,S}
13 H u0 p0 c0 {5,S}
14 H u0 p0 c0 {6,S}
15 H u0 p0 c0 {7,S}
16 H u0 p0 c0 {7,S}

iafter
1     C u0 p0 c0 {2,S} {3,S} {4,S} {7,S}
2     C u0 p0 c0 {1,S} {5,D} {9,S}
3     C u0 p0 c0 {1,S} {6,D} {10,S}
4  *1 C u0 p0 c0 {1,S} {8,D} {13,S}
5

In [ ]:
family = database.kinetics.families[fam_rxn.family]
family._KineticsFamily__matchReactantToTemplate

## get speciesDict

### load existing species as an intial speciesDict

In [ ]:
import os
from rmgpy.data.base import Database

training_path = os.path.join(settings['database.directory'], \
                             'kinetics', 'families', 'R_Addition_MultipleBond', 'training')

dictionary_file = os.path.join(training_path, 'dictionary.txt')

# Load the existing set of the species of the training reactions
speciesDict = Database().getSpecies(dictionary_file)
print speciesDict.keys()

### for one family check uniqueness of each species in the lib_rxns

In [ ]:
familyName = 'R_Addition_MultipleBond'
reactions = reactionDict[familyName]

for rxn in reactions:
    if rxn.index in [1,2]:
        print 'before'
        print rxn.reactants[0].toAdjacencyList()
        for spec in (rxn.reactants + rxn.products):
            if spec.molecule[0].getFormula() == 'C7H8':
                print spec.toAdjacencyList()
                for ex_spec_label in speciesDict:
                    ex_spec = speciesDict[ex_spec_label]
                    if ex_spec.molecule[0].getFormula() != spec.molecule[0].getFormula():
                        continue
                    else:
                        spec_labeledAtoms = spec.molecule[0].getLabeledAtoms()
                        print '#' + str(rxn.index)
                        print spec.toAdjacencyList()
                        print spec_labeledAtoms
                        ex_spec_labeledAtoms = ex_spec.molecule[0].getLabeledAtoms()
                        print '# the other'
                        print ex_spec.toAdjacencyList()
                        print ex_spec_labeledAtoms
                        initialMap = {}
                        try:
                            for atomLabel in spec_labeledAtoms:
                                initialMap[spec_labeledAtoms[atomLabel]] = ex_spec_labeledAtoms[atomLabel]
                        except KeyError:
                            # atom labels did not match, therefore not a match
                            continue
                        # loop each molecule
                        if spec.molecule[0].isIsomorphic(ex_spec.molecule[0],initialMap):
                            print 'wrong should not be isomorphic!'
                            spec.label = ex_spec.label
#                             if spec.molecule[0].getFormula() == 'C7H8':
#                                 print spec
#                                 print rxn.index
#                                 print spec_labeledAtoms
#                                 print spec.molecule[0].toAdjacencyList()
                            break
                else:# no isomorphic existing species found
                    spec_formula = spec.molecule[0].getFormula()
                    if spec_formula not in speciesDict:
                        spec.label = spec_formula
                    else:
                        index = 2
                        while (spec_formula + '-{}'.format(index)) in speciesDict:
                            index += 1
                        spec.label = spec_formula + '-{}'.format(index)
                    speciesDict[spec.label] = spec

In [ ]:
print speciesDict['C7H8'].toAdjacencyList()

In [ ]:
speciesDict['C7H8'].molecule[0].getLabeledAtoms()